In [2]:
from pynq import Overlay
from pynq.lib.iic import *
from ov5640_config import *
from pynq.lib.video import *
from matplotlib import pyplot as plt
from time import sleep
import pynq.lib.dma
from pynq import allocate
import cv2
import os
import shutil

## Instantiate Overlay and download bitstream

In [3]:
ov5640_mnist = Overlay("./../bitstreams/ov5640_mnist.bit")

In [4]:
iic = AxiIIC(ov5640_mnist.ip_dict['axi_iic_0'])
address = 0x3c
length = 3

In [5]:
for config in ov5640_config:
    tmp1 = config[0] >> 8;
    tmp2 = config[0] & 0xff;
    iic.send(address, bytes([tmp1, tmp2, config[1]]), length)

In [6]:
bayer2rgb = ov5640_mnist.v_demosaic_0
bayer2rgb.write(0x10, 1280)
bayer2rgb.write(0x18, 720)
bayer2rgb.write(0x28, 1)
bayer2rgb.write(0x00, 0x81)

# Check the attributes and methods in Overlay

In [7]:
ov5640_mnist.ip_dict

{'axi_dma_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb0205eb0>,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S_AXI_LITE',
  'parameters': {'C_BASEADDR': '0x40400000',
   'C_DLYTMR_RESOLUTION': '125',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_FAMILY': 'zynq',
   'C_HIGHADDR': '0x4040FFFF',
   'C_INCLUDE_MM2S': '1',
   'C_INCLUDE_MM2S_DRE': '0',
   'C_INCLUDE_MM2S_SF': '1',
   'C_INCLUDE_S2MM': '1',
   'C_INCLUDE_S2MM_DRE': '0',
   'C_INCLUDE_S2MM_SF': '1',
   'C_INCLUDE_SG': '0',
   'C_INCREASE_THROUGHPUT': '0',
   'C_MICRO_DMA': '0',
   'C_MM2S_BURST_SIZE': '16',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_M_AXIS_MM2S_TDATA_WIDTH': '32',
   'C_M_AXI_MM2S_ADDR_WIDTH': '32',
   'C_M_AXI_MM2S_DATA_WIDTH': '64',
   'C_M_AXI_S2MM_ADDR_WIDTH': '32',
   'C_M_AXI_S2MM_DATA_WIDTH': '32',
   'C_M_AXI_SG_ADDR_WIDTH': '32',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2MM

## Configure VDMAs

In [8]:
vdma0 = ov5640_mnist.axi_vdma_0

In [9]:
hex(ov5640_mnist.ip_dict["axi_vdma_0"]["phys_addr"])

'0x43000000'

In [10]:
dma0 = ov5640_mnist.axi_dma_0

In [11]:
hex(ov5640_mnist.ip_dict["axi_dma_0"]["phys_addr"])

'0x40400000'

In [12]:
mode = VideoMode(1280, 720, 24)
vdma0.readchannel.mode = mode
vdma0.writechannel.mode = mode

In [15]:
vdma0.readchannel.tie(vdma0.writechannel)
vdma0.writechannel.start()
vdma0.readchannel.start()

In [16]:
output_buf = allocate(shape=(1,), dtype=np.uint32)

In [33]:
import time

while(1):
    start = time.time()
    dma0.recvchannel.transfer(output_buf)
    dma0.recvchannel.wait()
    end = time.time()
    if(round(end-start,5) > 1):
        print('time cost: ' + str(round(end-start,5)) + ' s')
        print(str(output_buf[0]))
        sleep(1)

time cost: 2.40009 s
1
time cost: 1.3982 s
1
time cost: 1.39821 s
1
time cost: 1.39821 s
5
time cost: 1.39822 s
0
time cost: 1.39802 s
0
time cost: 1.39824 s
0


KeyboardInterrupt: 

## Close VDMAs

In [34]:
vdma0.writechannel.stop()
vdma0.readchannel.stop()